In [ ]:
%matplotlib inline
import time
from pymongo import MongoClient 
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from datetime import datetime

In [ ]:
hostip = "10.192.41.222"
client = MongoClient(
        host=f'{hostip}',
        port=27017
    )
db = client["awas_db"]
violation_col = db["violation"]

print(violation_col.count_documents({}))

for doc in violation_col.find().limit(5):
    print(doc)

In [ ]:
# Convert MongoDB timestamp to pandas datetime
def fetch_violation_data():
    pipeline = [
        {"$unwind": "$violations"},
        {"$project": {
            "_id": 0,
            "timestamp": "$violations.timestamp_start"
        }}
    ]
    cursor = violation_col.aggregate(pipeline)
    timestamps = [doc["timestamp"] for doc in cursor if "timestamp" in doc]
    return pd.DataFrame(timestamps, columns=["timestamp"])

In [ ]:
# Real-time plotting function
def visualize_stream(interval_sec=5):
    # fig, ax = plt.subplots(figsize=(12, 6))

    while True:
        df = fetch_violation_data()
        if df.empty:
            print("No data yet...")
            time.sleep(interval_sec)
            continue

        # Convert string/object to datetime
        df["timestamp"] = pd.to_datetime(df["timestamp"], format='ISO8601', errors='coerce')

        # Group by 10-second intervals
        df.set_index("timestamp", inplace=True)
        counts = df.resample("5s").size()

        clear_output(wait=True)
        plt.figure(figsize=(12, 6))
        plt.plot(counts.index, counts.values, marker='o', linestyle='-')

        if not counts.empty:
            max_time = counts.idxmax()
            max_val = counts.max()
            min_time = counts.idxmin()
            min_val = counts.min()
            plt.annotate(f'Max: {max_val}', xy=(max_time, max_val), xytext=(max_time, max_val + 1),
                         arrowprops=dict(facecolor='green'), fontsize=10)
            plt.annotate(f'Min: {min_val}', xy=(min_time, min_val), xytext=(min_time, min_val + 1),
                         arrowprops=dict(facecolor='red'), fontsize=10)

        plt.title("Violations Over Time (10s)")
        plt.xlabel("Time")
        plt.ylabel("Number of Violations")
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        time.sleep(interval_sec)

if __name__ == "__main__":
    visualize_stream()
